# Development Backtesting Enviornment
---
We want to develope an Environment to test & create different strategies as well as evaluate parameter for strategies.



1. Load Candles for testing date range

In [3]:
from Backtester.streamer.candles import CandlesInterface
from Backtester.time.time_utility import to_tms

candles = CandlesInterface()
data = candles.get("bybit", "BTC/USDT:USDT", "1m", start="2022-10-20", end="2022-12-09")

{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1666224000000, 'end': 1670544000000, 'limit': 72000}
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1666224000000, 'end': 1670544000000, 'limit': 72000} 199
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1666224000000, 'end': 1670544000000, 'limit': 72000} 398
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1666224000000, 'end': 1670544000000, 'limit': 72000} 597
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1666224000000, 'end': 1670544000000, 'limit': 72000} 796
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1666224000000, 'end': 1670544000000, 'limit': 72000} 995
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1666224000000, 'end': 1670544000000, 'limit': 72000} 1194
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m

2. Indicator

gather from db: {'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m'}
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '15m', 'end': 1664582400000, 'limit': 500, 'start': 1664132400000} 199
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1h', 'end': 1664582400000, 'limit': 500, 'start': 1662782400000} 199
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '15m', 'end': 1664582400000, 'limit': 500, 'start': 1664132400000} 398
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1h', 'end': 1664582400000, 'limit': 500, 'start': 1662782400000} 398
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '15m', 'end': 1664582400000, 'limit': 500, 'start': 1664132400000} 500
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1h', 'end': 1664582400000, 'limit': 500, 'start': 1662782400000} 500


In [1]:
from Backtester.indicators.wave_cluster import get_wave_reports
from Backtester.indicators.waves import IterWaves, WaveIterator
from Backtester.streamer.candles import streamer



def update_data(current_tms: int):
    return get_wave_reports(streamer.gather("bybit", "BTC/USDT:USDT", ["1m", "15m", "1h"], end=current_tms, limit=500))

reports = update_data(1666224060000)
reports

gather from db: {'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m'}
gather from db: {'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '15m'}
gather from db: {'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1h'}


[({'exchange': 'bybit',
   'market': 'BTC/USDT:USDT',
   'timeframe': '1m',
   'end': 1666224060000,
   'limit': 500,
   'start': 1666194060000},
  {-2: 19158.5, -3: 19129.5, -4: 19075.5, 2: 19220.5, 3: 19252.0, 4: 19296.5}),
 ({'exchange': 'bybit',
   'market': 'BTC/USDT:USDT',
   'timeframe': '15m',
   'end': 1666224000000,
   'limit': 500,
   'start': 1665774000000},
  {-2: 19311.5, -3: 19155.0, -4: 18962.0, 2: 19374.0, 3: 19483.0, 4: 19694.0}),
 ({'exchange': 'bybit',
   'market': 'BTC/USDT:USDT',
   'timeframe': '1h',
   'end': 1666224000000,
   'limit': 500,
   'start': 1664424000000},
  {-2: 19028.0, -3: 18831.0, -4: 18071.0, 2: 19799.0, 3: 19982.5, 4: 20451.0})]

Backtest how likely the given entires/exits are met.

### PriceLevelIndicator

In [2]:
waves = WaveIterator("buy", reports, 19299)
waves.iter.data

{('1h', -4): 18071.0,
 ('1h', -3): 18831.0,
 ('15m', -4): 18962.0,
 ('1m', -4): 19075.5,
 ('1m', -3): 19129.5,
 ('1m', -2): 19158.5,
 ('1m', 2): 19220.5,
 ('1m', 3): 19252.0,
 ('1m', 4): 19296.5,
 ('15m', -2): 19311.5,
 ('15m', 2): 19374.0,
 ('15m', 3): 19483.0,
 ('15m', 4): 19694.0,
 ('1h', 2): 19799.0,
 ('1h', 3): 19982.5,
 ('1h', 4): 20451.0}

In [6]:
import numpy as np
from enum import IntEnum
from abc import ABC, abstractmethod

EntryExitPrice = tuple[float, float]

class candle(IntEnum):
    TMS = 0
    OPEN = 1
    HIGH = 2
    LOW = 3
    CLOSE = 4
    VOLUME = 5



class BacktesterPriceLevels:
    def __init__(self, meta: dict, data: list):
        self.meta = meta
        self.data = np.array(data)
        self.i = 0

        self._indicator = None
        self._indicator_method = None
        self._side = None
        self._state = None
        self._bets = []
        self._entry = None
        self._exit = None

    @property
    def price(self):
        return self.data[self.i, candle.CLOSE]

    @property
    def timestamp(self):
        return self.data[self.i, candle.TMS]

    def next(self):
        self.i += 1

    def add_indicator(self, indicator_object: PriceLevelIndicatorIter, indicator_method) -> None:
        self._indicator = indicator_object
        self._indicator._method = indicator_method

    def get_levels(self):
        entry, exit = self._indicator.next()
        self._entry = entry
        self._exit = exit

    def reset_indicator(self) -> None:
        self._indicator.reset(get_wave_reports(streamer._gather(self._indicator.get_reset_queries(self.timestamp))), self.price)






In [7]:
benv = BacktesterPriceLevels(*data)

In [8]:
benv.add_indicator(waves, get_wave_reports)

In [9]:
benv.timestamp

1666224060000.0

In [10]:
benv._indicator_method(streamer.gather("bybit", "BTC/USDT:USDT", ["1m", "15m", "1h"], end=int(1666224060000), limit=500))

gather from db: {'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m'}
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '15m', 'end': 1666224000000, 'limit': 500, 'start': 1665774000000} 199
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1h', 'end': 1666224000000, 'limit': 500, 'start': 1664424000000} 199
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '15m', 'end': 1666224000000, 'limit': 500, 'start': 1665774000000} 398
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1h', 'end': 1666224000000, 'limit': 500, 'start': 1664424000000} 398
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '15m', 'end': 1666224000000, 'limit': 500, 'start': 1665774000000} 500
{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1h', 'end': 1666224000000, 'limit': 500, 'start': 1664424000000} 500


[({'exchange': 'bybit',
   'market': 'BTC/USDT:USDT',
   'timeframe': '1m',
   'end': 1666224060000,
   'limit': 500,
   'start': 1666194060000},
  {-2: 19158.5, -3: 19129.5, -4: 19075.5, 2: 19220.5, 3: 19252.0, 4: 19296.5}),
 ({'exchange': 'bybit',
   'market': 'BTC/USDT:USDT',
   'timeframe': '15m',
   'end': 1666224000000,
   'limit': 500,
   'start': 1665774000000},
  {-2: 19311.5, -3: 19155.0, -4: 18962.0, 2: 19374.0, 3: 19483.0, 4: 19694.0}),
 ({'exchange': 'bybit',
   'market': 'BTC/USDT:USDT',
   'timeframe': '1h',
   'end': 1666224000000,
   'limit': 500,
   'start': 1664424000000},
  {-2: 19028.0, -3: 18831.0, -4: 18071.0, 2: 19799.0, 3: 19982.5, 4: 20451.0})]

In [13]:
candle.CLOSE

<candle.CLOSE: 4>

In [4]:
waves.get_reset_queries(123)

[{'exchange': 'bybit',
  'market': 'BTC/USDT:USDT',
  'timeframe': '1m',
  'limit': 500,
  'end': 123},
 {'exchange': 'bybit',
  'market': 'BTC/USDT:USDT',
  'timeframe': '15m',
  'limit': 500,
  'end': 123},
 {'exchange': 'bybit',
  'market': 'BTC/USDT:USDT',
  'timeframe': '1h',
  'limit': 500,
  'end': 123}]